In [1]:
import torch
from torch import nn
from kornia.filters import laplacian, sobel
from torch.utils.data import Dataset
from torchvision.io import read_image

In [2]:
'''
SIREN needs a specific way of initialization for the
Layers
#code inspired from youtuber mildlyoverfitted
'''

def initializer(weight: torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor,
                is_first: bool = True,
                omega: float = 1.,
                c:flaot = 6.
               ):
    '''
    Intializes the wrights of linear layer
    Weights are based on the paper(supplimentary page 5) presented by SIREN, choose the distribution in range
    For first layer:
        uniform distribution in the range: (-1, 1):
    For rest of layers:
        bound = sqrt(c / (features_in * (omega**2)))
        uniform distribution in the range: [-bound, bound]
        
    params:
            weight: torch.FloatTensor
            ----------------> Initialzing weight for Linear layer
            is_first: bool
            ----------------> If true, the layer is first layer in the network
            omega: float
            ----------------> hyperparameter
            c: float
            ---------------->hyperparameter
    '''
    
    with torch.no_grad():
        bound = 1 / weight.size()[1] if is_first else torch.sqrt(c / weight.size()[1]) / omega
        weight.uniform_(-bound, bound)
    

In [3]:
class siren_layer(nn.Module):
    '''
    SIREN LAYER - Sinusoidal Representation Network Layer
    params:
        in_features: int
            #input_features
        ------------------
        out_features: int
            #out_features
        ------------------
        bias: bool
            True->Needs Bias ; False->Do not Need Bias
        ------------------
        is_first: bool
            True->Is the fisr layers; False->Not First Layer
            Has influence on the initialization of the layer
        ------------------
        omega: flaot
            Hyperparameter helpful in initialization
        ------------------
        c: int
            Hyperparameter helful in initialization
        ------------------
        custom_initalizationg_function: function
            Send custom initialization function accepting a Tensor for initializing it        
    '''
    def __init__(
                    self, 
                    in_features: int,
                    out_features: int,
                    bias: bool = True,
                    is_first: bool = False,
                    omega: float = 30,
                    c: float = 6,
                    custom_initalizationg_function: callable = None
                ):
        super(siren_layer).__init__()
        
        self.omega, self.c = omega, c
        
        self.linear = nn.Linear(
                                    in_features = in_features,
                                    out_features = out_features,
                                    bias = bias
                                )
        initializer(self.linear.weight, is_first, omega, c) if custom_initalizationg_function is None else custom_initalizationg_function(self.linear.weight)
    
    def forward(self, x):
        '''
        sin( omega * Linear(input))
        '''
        ##### LOOK AT THIS LATER #####
        return torch.sin(self.omega  * torch.dot(self.linear.weight, x) + self.linear.bias)

In [8]:
class image_siren(nn.Module):
    '''
    
    '''

AttributeError: module 'torch.cuda' has no attribute 'Tensor'